In [ ]:
import numpy as np
import ymmsl
import yaml
import sobol_seq
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns
import zenodo_get

In [ ]:
# Transform the normalized sample matrix to ranges of uncertain parameters
def dim_transform(sobel_vector,uncertain_list):
    
    dim = len(uncertain_list)
    for num_dim in range(dim):
        para_max = uncertain_list[num_dim].get('max')
        para_min = uncertain_list[num_dim].get('min')
        sobel_vector[:,num_dim] = para_min + (para_max-para_min)*sobel_vector[:,num_dim]
        
    return sobel_vector

In [ ]:
####################################################################################
##### Sample generation and UQ campaign creation (including instances folder)#######
####################################################################################
# Note:
#  This is used to generate UQ samples for only four biological parameters:
#	1) Endothelium endpoint 2)smc max stran 3)balloon extension 4) Fenestration probability

# Naming of Folder and files for samples
# Level 0:    UQtest                                    (UQ campaign name)
# Level 1:    UQtest/A                                  (sample matrix of sobol sequence)
# Level 2:    UQtest/A/A_X where X vary from 1 -> N     (N: number of samples)
# Level 3:    UQtest/A/A_X/input.ymmsl


In [ ]:
### Main function
# Number of samples for UQ
# Note that ISR3D is a computationally intensive application.
# Running 128 instances would need some cluster resources
N_sample = 128

# Template path to the ymmsl file (relative path from ISR3D/Result/UQtest/ to ISR3D/UQ/template/input_stage4.ymmsl)
input_path = '../../UQ/template/'
input_ymmsl_filename = 'input_stage4.ymmsl'

# Output directory for UQ campagin folder and name
output_path = './'
experiment_name = 'UQtest'

In [ ]:
# Read in the data of template ymmsl file
with open(input_path+input_ymmsl_filename,'r') as f:
    ymmsl_data = ymmsl.load(f)


In [ ]:
# Take out the unchanged model part and need-for-change settings part for ymmsl model
model = ymmsl_data.model
settings = ymmsl_data.settings

In [ ]:
# Set uncertain parameters and its ranges as a list
ymmsl_uncertain_parameters = [
        {
            'name': 'smc.endo_endpoint',
            'min': 10.0,
            'max': 20.0
        },
        {
            'name': 'smc.balloon_extension',
            'min': 0.5,
            'max': 1.5
        },
        {
            'name': 'smc.smc_max_strain',
            'min': 1.2,
            'max': 1.8
        },
        {
            'name': 'smc.fenestration_probability',
            'min': 0.0,# Calculate the lumen volume from (lumen_area_of_each_slice*depth_of_slice)
            'max': 0.1
        }]

In [ ]:
# Count the total uncertain input dimensions (here 4 parameters)
num_uncer_para = len(ymmsl_uncertain_parameters)
print('Number of uncertain parameter: '+str(num_uncer_para))

In [ ]:
# Generate sobel sequence range (0,1), save the file and transform to (min,max)
A = sobol_seq.i4_sobol_generate(num_uncer_para,N_sample)
A = dim_transform(A,ymmsl_uncertain_parameters)
np.savetxt("A.csv",A)

In [ ]:
# Create corresponding directory and folders
try:
    os.mkdir(output_path+experiment_name)
except OSError:
    print ("Creation of the directory %s failed" % output_path+experiment_name)
else:
    print ("Successfully created the directory %s" % output_path+experiment_name)

In [ ]:
# A: Replace the corresponding value within the dict and output the file
os.mkdir(output_path+experiment_name+'/A')
checklist = ['A']

for n in range(N_sample):
    sample_path = output_path+experiment_name+'/A'+'/A_'+str(n)
    os.mkdir(sample_path)
    
    # Generate file for ymmsl 
    num_para = 0
    for para in ymmsl_uncertain_parameters:
        settings[para.get('name')] = float(A[n,num_para])
        num_para = num_para + 1
        
    config = ymmsl.Configuration(model, settings)
    with open(sample_path+'/input_stage4.ymmsl', 'w') as f:
        ymmsl.save(config, f)

print('Wrting and saving procedure done: A')

In [ ]:
####################################################################################
##### Run shell script to broadcast other input files to each sample folder#########
####################################################################################

In [ ]:
import subprocess
# Download Other input files from Zenodo
subprocess.run(['wget https://zenodo.org/record/4603912/files/stage3.test_vessel.dat'],shell = True)
subprocess.run(['wget https://zenodo.org/record/4603912/files/stage3.test_vessel_nb.dat'],shell = True)
subprocess.run(['wget https://zenodo.org/record/4603912/files/test_vessel_centerline.csv'],shell = True)

# Template path to the ymmsl file (relative path from ISR3D/Result/UQtest/ to ISR3D/UQ/function/BCastStage3.sh)
pass_arg = str(N_sample)
subprocess.run(['bash','../../UQ/function/BCastStage3.sh', '%s'%pass_arg]) 

In [ ]:
####################################################################################################
##### You can copy the slurm job array to the directory and launch through terminal ################
##### Or directly do it here                                                        ################
####################################################################################################

In [ ]:
# This slurm script is made for the execution on Eagle supercomputer from PSNC. 
# To run the job array on your own cluster, you would need to tweak the setting a bit
# For example. partition=XXXX ,
subprocess.run(["sbatch", "../../UQ/slurm/job_array.sl"])

In [ ]:
################################################################################
###### Fetch the result data and plot out the PDF and mean+/-STD figures #######
################################################################################

In [ ]:
# Function of reading data
def LumenRead(path,numfile):
    
    resultlist = np.empty((0,39), float)
    for i in range(numfile):
        filename = os.path.join(path,('lumen_area_000'+str("{0:0=3d}".format(i))+'.csv'))
        reader = csv.reader(open(filename, "r"), delimiter='\t')
        x = list(reader)
        result = np.array(x[0][:-1]).astype("float")
        resultlist = np.append(resultlist, np.expand_dims(result,axis=0), axis=0)
        
    # print(resultlist.shape)
    return resultlist

In [ ]:
# Set Directory and number of instance in the UQ campaign
data_root = './UQtest/A/'
time_step = 361

# Read the all subdirectory of UQ instances in a list
data_list = [os.path.join(data_root, item) for item in sorted(os.listdir(data_root))]

#Create an empty list and fetch the data in an loop
lumen_list = np.empty((0,time_step,39), float)
for item in data_list:
    print('Processing:',item) 
    Data = LumenRead(item,time_step)
    lumen_list = np.append(lumen_list,np.expand_dims(Data,axis=0),axis=0)

print(lumen_list.shape)	
np.save('LumenData',lumen_list)

In [ ]:
# Calculate the lumen volume from (lumen_area_of_each_slice*depth_of_slice)
lumen_list = np.load('LumenData.npy')
lumen_vol = np.sum(lumen_list[:,:,:],axis=2) * 0.03125
fig = plt.figure()

# plt.plot(np.ones(128)*3.56055,np.linspace(0.0,2.5,128),label='0 days',c='k')
sns.kdeplot(lumen_vol[:,72],label='3 days',fill=True)
sns.kdeplot(lumen_vol[:,144],label='6 days',fill=True)
sns.kdeplot(lumen_vol[:,216],label='9 days',fill=True)
sns.kdeplot(lumen_vol[:,288],label='12 days',fill=True)
sns.kdeplot(lumen_vol[:,360],label='15 days',fill=True)
plt.legend(fontsize=10,loc=2)
plt.xlabel('Lumen volume of blood vessel ($mm^3$)',fontsize=12)
plt.ylabel('Probability density function',fontsize=12)
plt.savefig('./'+'pdf.png')
plt.show()

In [ ]:
# plot mean+/-STD
days = np.zeros(361)
for i in range(361):
    days[i] = i/24

plt.plot(days, np.mean(lumen_vol,axis=0),label='Mean')
plt.plot(days, np.mean(lumen_vol,axis=0) - np.std(lumen_vol,axis=0), '--',color='r',label='Mean+/-STD')
plt.plot(days, np.mean(lumen_vol,axis=0) + np.std(lumen_vol,axis=0), '--',color='r')
plt.xlabel('Time (days)')
plt.ylabel('Lumen volume of blood vessel ($mm^3$)')
plt.legend()
plt.savefig('./'+'mean.png')
plt.show()
